In [1]:
import pickle
import numpy as np

In [2]:
ratings = pickle.load(open('items_10000_matrix.pickle', 'rb'), encoding='iso-8859-1')

In [3]:
ratings.shape

(71869, 10000)

In [4]:
type(ratings)

scipy.sparse.csr.csr_matrix

In [5]:
ratings = ratings.toarray()

In [6]:
ratings.shape

(71869, 10000)

*ratings* -  np.array состоящий из нулей и единиц, где каждая строка соответсвует пользователю, а каждая колонка - книге.  
1 - на пересечении строки i колонки j значит, что пользователь i прочитал книгу j. 0 - пользователь i не читал книгу j.

# Part 1

### Task 1. Найти самую популярную книгу.

Сложим все строки между собой - получим количество пользователей, прочитавших каждую книгу
Затем применим np.argmax к полученному вектору для получения самой популярной книги

In [7]:
# Пример: для сложения элементов строк нужно указать axis=0
np.array([
    [1,2],
    [3,4]
]).sum(axis=0)

array([4, 6])

In [8]:
most_popular_book = np.argmax(ratings.sum(axis=0))
print('Most popular book = {}'.format(most_popular_book))

Most popular book = 20


### Task 2. Найти пользователя со второй по величине библиотекой.

In [9]:
# Пример: для сложения элементов столбцов нужно указать axis=1
# Затем сортируем массив при помощи argsort и получем второй элемент
test = np.array([
    [1,2],
    [3,4],
    [10, 16],
    [4, 0]
]).sum(axis=1)
test_ids = test.argsort()
print(f"Second largest index: {test_ids[-2]}")
test

Second largest index: 1


array([ 3,  7, 26,  4])

In [10]:
amount_books_per_user = ratings.sum(axis=1)

second_biggest_library = amount_books_per_user.argsort()[-2]
print('user_id with the second biggest library = {}'.format(second_biggest_library))

user_id with the second biggest library = 69648


### Task 3. Найти среднее и медиану размера библиотеки.

In [11]:
mean = amount_books_per_user.mean()
print('mean = {}'.format(mean))

median = np.median(amount_books_per_user)
print('median = {}'.format(median))

mean = 2.266693567463023
median = 2.0


## Ответ к заданию: 2

### Task 4. Найти медиану размера библиотеки у пользователей, которые прочитали книгу номер с id 100.

In [12]:
median_number = np.median(ratings[ratings[:, 100] == 1].sum(axis=1))
print('Median number of books of users who read the book with id 100 = {}'.format(median_number))

Median number of books of users who read the book with id 100 = 3.0


# Part 2

### Task 1. Похожие книги.
1. Найди 20 наиболее похожих книг для книги с id = 100. В качестве меры похожести используй cosine similarity. Используй наиболее быстрый вариант подсчета.
  
$$cosine\ similarity = \frac{A\cdot B}{||A||\ ||B||} = \frac{\sum_{i=1}^{n} A_iB_i}{\sqrt{\sum_{i=1}^{n}A_i^2}\sqrt{\sum_{i=1}^{n}B_i^2}}$$  
  
В качестве ответа верни список кортежей, где каждый кортеж представляет собой пару (book_id, similarity).

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

In [15]:
a = csr_matrix(ratings.T)
b = csr_matrix([ratings[:, 100]])

sim = cosine_similarity(a,b).flatten()
most_similar_20_indices = sim.argsort()[-2:-22:-1] # выкидываем первое число, т.е. сам документ c id = 100
most_similar_20 = [(idx, sim[idx]) for idx in most_similar_20_indices]

In [16]:
most_similar_20

[(5863, 0.2672612419124244),
 (5819, 0.2672612419124244),
 (5134, 0.1889822365046136),
 (2370, 0.1336306209562122),
 (3940, 0.1336306209562122),
 (5440, 0.1336306209562122),
 (3468, 0.10910894511799621),
 (5546, 0.08451542547285167),
 (1495, 0.06482037235521644),
 (214, 0.06482037235521644),
 (6, 0.04879500364742666),
 (528, 0.0445435403187374),
 (251, 0.034794450031961056),
 (836, 0.031943828249996996),
 (426, 0.03065696697424829),
 (312, 0.03026137663344012),
 (446, 0.025141822757713456),
 (73, 0.02126216277812811),
 (191, 0.016671298226202138),
 (76, 0.01566713146512808)]

# Part 3

### Task 1. Матрица похожих книг (топ-20).

Создать матрицу похожих книг (матрица 10000 x 10000) использую метрику *cosine similarity*. В каждоый строке оставить только топ 20 самых похожих. Лучше написать функцию которая на вход получает матрицу рейтингов на выходе матрица похожих книг. Эту фунцкцию можно будет использовать в следующем задании.

In [17]:
def compute_sim_matrix(ratings):
    similariry_matrix = cosine_similarity(csr_matrix(ratings).T).argsort(axis=1)
    return similariry_matrix[:,-2:-22:-1]

In [18]:
similariry_matrix = compute_sim_matrix(ratings)

In [19]:
similariry_matrix.shape

(10000, 20)

### Task 2. Оценить рекомендательную систему.

Оценить качество рекомендательной системы, основанной на матрице похожих книг, использую метрики **precision** и **recall**. Рекомендации должны получаться путем умножения матрицы похожих книг на вектор пользователя.
  
**precison** - кол-во книг которые попали пользователю в рекомендации и он их действительно добавил делить на кол-во рекомендованных книг.
  
**recall** - кол-во книг которые попали пользователю в рекомендации и он их действительно добавил делить на кол-во книг добавленных пользователем. 
  
Метрики считать на 20 самых лучших рекомендациях для каждого пользователя.

In [20]:
ratings[0].shape

(10000,)

In [21]:
ratings[0]

array([0, 0, 0, ..., 0, 0, 0])

In [31]:
similariry_matrix

array([[8168, 9529,    2, ...,  990,  167,  394],
       [  41,   64, 2125, ..., 7499, 3713, 4468],
       [  50,    0,   11, ...,  493, 9494, 8678],
       ...,
       [3329, 3336, 3335, ..., 3337, 3339, 3436],
       [3329, 3336, 3335, ..., 3337, 3339, 3436],
       [3329, 3336, 3335, ..., 3337, 3339, 3436]])

In [25]:
ratings[0].dot(similariry_matrix)

array([  29,   34,  335,  335,  408,  654,  320,  140,  320,   83,  614,
        105, 1866,  285,   31,   88,   49,  133,  386,  524])

In [28]:
ratings[0][ratings[0].dot(similariry_matrix)]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])